In [5]:
import os
import json
import pandas as pd
import traceback

In [6]:
pip install langchain
pip install langchain-community


  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.chat_models import ChatOpenAI


In [10]:
llm=ChatOpenAI(openai_api_key=(os.getenv("OPENAI_API_KEY")), model='gpt-3.5-turbo', temperature=0.5)

C:\Users\sarik\AppData\Local\Temp\ipykernel_23600\4126121086.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm=ChatOpenAI(openai_api_key=(os.getenv("OPENAI_API_KEY")), model='gpt-3.5-turbo', temperature=0.5)


In [12]:
llm.predict("who was the first prime minister of India?")

C:\Users\sarik\AppData\Local\Temp\ipykernel_23600\2081127022.py:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  llm.predict("who was the first prime minister of India?")


'Jawaharlal Nehru was the first Prime Minister of India. He served as Prime Minister from 1947 to 1964.'

In [14]:
# other way to get a key is to create .env file and use load_dotenv method to read a key.

from dotenv import load_dotenv
load_dotenv()
key=os.getenv("openai_apikey")
print(key)

sk-yLfIMw0nZERL82fvH2F5c0oni8Uda7OGHBjgOeWJY1T3BlbkFJjCciNB3_u5lYLhMQurkm8HHdd2OBvQN0y-BerIIgcA


In [15]:
llm=ChatOpenAI(openai_api_key=key, model='gpt-3.5-turbo', temperature=0.5)
llm.predict("which all companies owned by Elon musk")

'1. Tesla Inc.\n2. SpaceX\n3. The Boring Company\n4. Neuralink\n5. SolarCity (acquired by Tesla)\n6. OpenAI (co-founded by Musk)'

In [16]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [19]:
response_json = {
    "1": {
        "mcq": "question",
        "options": {
            "a": "Option 1",
            "b": "Option 2",
            "c": "Option 3",
            "d": "Option 4",
            "e": "Option 5"
        },
        "correct_answer": "correct answer"
    }
}


In [20]:
TEMPLATE=""" 
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz \
of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check if all the questions are conforming 
the text. Please format the questions in response_json format given below.
please create {number} questions.

### response_json
{response_json}
"""

In [21]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [22]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose=True)

C:\Users\sarik\AppData\Local\Temp\ipykernel_23600\1958558231.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose=True)


In [23]:
TEMPLATE2=""" 
You are an expert english grammarian and writer. Given a multiple choice quiz for {subject} students,
you need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max
50 words for the complexity. If the quiz is not as per cognitive and analytical ability of the students, 
update the quiz which needs to be changed and change the tone so that it fits perfectly with student.

Quiz MCQs:
{quiz}

check from an expert English writer for the above quiz.
"""

In [24]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2

)

In [27]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [30]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz", "review"],
    verbose=True
    
    )

In [34]:
# Read the data from data.txt file
file_path = r"C:\Users\sarik\mcqgen\mcqgenerator.egg-info\data.txt"

try:
    with open(file_path, "r", encoding="utf-8") as file:
        TEXT = file.read()
    print("File read successfully!")
except FileNotFoundError:
    print(f"Error: The file at {file_path} was not found.")
except UnicodeDecodeError:
    print(f"Error: Unable to decode the file. Please check the encoding.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


File read successfully!


In [35]:
print(TEXT)

What is machine learning?
Machine learning (ML) is a branch of artificial intelligence (AI) focused on enabling computers and machines to imitate the way that humans learn, to perform tasks autonomously, and to improve their performance and accuracy through experience and exposure to more data.

UC Berkeleybreaks out the learning system of a machine learning algorithm into three main parts.

A Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.

An Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.

A Model Optimization Process: If the model can fit better to the data points in the training set, then weights are adjusted to reduce the discrepancy between the known example and th

In [47]:
NUMBER=5,
SUBJECT="Machine Learning"
TONE="simple"

In [48]:
#How to setup token usage tracking in Langchain
# https://python.langchain.com/docs/how_to/llm_token_usage_tracking/

with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text" : TEXT,
            "number" : NUMBER,
            "subject" : SUBJECT,
            "tone" : TONE,
            "response_json" : json.dumps(response_json) 

        }


    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
 
Text:What is machine learning?
Machine learning (ML) is a branch of artificial intelligence (AI) focused on enabling computers and machines to imitate the way that humans learn, to perform tasks autonomously, and to improve their performance and accuracy through experience and exposure to more data.

UC Berkeleybreaks out the learning system of a machine learning algorithm into three main parts.

A Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.

An Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.

A Model Optimization Process: If the model can fit better to the data points in the training set, then weights are adjusted to reduce the discrepancy 

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:
 
You are an expert english grammarian and writer. Given a multiple choice quiz for Machine Learning students,
you need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max
50 words for the complexity. If the quiz is not as per cognitive and analytical ability of the students, 
update the quiz which needs to be changed and change the tone so that it fits perfectly with student.

Quiz MCQs:
{
    "1": {
        "mcq": "What is the main focus of machine learning?",
        "options": {
            "a": "Imitating human emotions",
            "b": "Performing tasks autonomously",
            "c": "Creating new languages",
            "d": "Improving physical strength"
        },
        "correct_answer": "b"
    },
    "2": {
        "mcq": "Which type of learning uses labeled datasets to train algorithms?",
        "options": {
            "a": "Supervised learning",
            "b": "Unsupervised

In [49]:
response

{'text': 'What is machine learning?\nMachine learning (ML) is a branch of artificial intelligence (AI) focused on enabling computers and machines to imitate the way that humans learn, to perform tasks autonomously, and to improve their performance and accuracy through experience and exposure to more data.\n\nUC Berkeleybreaks out the learning system of a machine learning algorithm into three main parts.\n\nA Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.\n\nAn Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.\n\nA Model Optimization Process: If the model can fit better to the data points in the training set, then weights are adjusted to reduce the discrepancy between the k

In [45]:
print("Total tokens :", cb.total_tokens)
print("Prompt tokens :", cb.prompt_tokens)
print("Completion tokens :", cb.completion_tokens)
print("Total cost :", cb.total_cost)


Total tokens : 4998
Prompt tokens : 4457
Completion tokens : 541
Total cost : 0.0077675


In [53]:
quiz=response.get("quiz")
quiz_response=json.loads(quiz)


In [55]:
# Flatten the JSON to create a DataFrame
flattened_data = []
for key, value in quiz_response.items():
    mcq = value['mcq']
    options = value['options']
    correct_answer = value['correct_answer']
    flattened_data.append({
        'Question ID': key,
        'Question': mcq,
        'Option A': options['a'],
        'Option B': options['b'],
        'Option C': options['c'],
        'Option D': options['d'],
        'Correct Answer': correct_answer
    })

# Create DataFrame
df = pd.DataFrame(flattened_data)

print(df)
df.to_csv("machine_learning_quiz.csv", index=False)

  Question ID                                           Question  \
0           1        What is the main focus of machine learning?   
1           2  Which type of learning uses labeled datasets t...   
2           3  What is the purpose of clustering algorithms i...   
3           4  Which algorithm is used for predicting categor...   
4           5  What is a common application of computer visio...   

                   Option A                       Option B  \
0  Imitating human emotions  Performing tasks autonomously   
1       Supervised learning          Unsupervised learning   
2  Predict numerical values  Classify data into categories   
3           Neural networks              Linear regression   
4   Automated stock trading                Fraud detection   

                                 Option C  \
0                  Creating new languages   
1                Semi-supervised learning   
2  Analyze and cluster unlabeled datasets   
3                     Logistic regress